# Plain-Text CNN

## Preparations

### options

In [1]:
%reload_ext autoreload
%autoreload 2
dir_data = '../data/power_consumption/'

### modules

In [9]:
import pandas as pd  # data mangling and transforming
import numpy as np  # handling vectors and matrices

import torch
import torch.nn.functional as F
import torch.nn as nn

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

Using TensorFlow backend.


### dataset

In [3]:
train_X = np.load(dir_data+'train_Xu.npy')
train_y = np.load(dir_data+'train_yu.npy')

In [4]:
print(train_X.shape)
print(train_y.shape)

(994, 7, 1)
(994, 7)


## Modeling

### Options

In [5]:
epochs = 20
batch_size = 4
n_timesteps = 7 
n_features = 1
n_outputs = 1
lr = 1e-07

In [13]:
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(n_outputs))
model.compile(loss='mse', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


In [17]:
model.get_config()

{'name': 'sequential_1',
 'layers': [{'class_name': 'Conv1D',
   'config': {'name': 'conv1d_1',
    'trainable': True,
    'batch_input_shape': (None, 7, 1),
    'dtype': 'float32',
    'filters': 16,
    'kernel_size': (3,),
    'strides': (1,),
    'padding': 'valid',
    'data_format': 'channels_last',
    'dilation_rate': (1,),
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'scale': 1.0,
      'mode': 'fan_avg',
      'distribution': 'uniform',
      'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'MaxPooling1D',
   'config': {'name': 'max_pooling1d_1',
    'trainable': True,
    'strides': (2,),
    'pool_size': (2,),
    'padding': 'valid',
    'data_format': 'channels_last'}},
  {'class_name': 'Flatte

In [ ]:
nn_model = nn.Sequential(
    nn.Conv1d(in_channels=7, out_channels=16, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool1d(kernel_size=2),
    
    
    nn.Linear(, 10),
    nn.Tanh(),
    nn.Linear(50, 50),
    nn.Sigmoid(),
    nn.Linear(50, 100),
    nn.Tanh(),
    nn.Linear(100, 1)
    )

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=1e-07)
loss_fn = F.mse_loss

In [ ]:
# Define a utility function to train the model
def fit(df, num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in df:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred, yb)
            # Perform gradient descent
            opt.zero_grad()
            loss.backward()
            opt.step()
        if epoch%10==0:
            print("Epoch %s: Loss %s"%(epoch, loss.data.item()))
    print('Training loss: ', loss_fn(model(inputs), target).data.item())

In [ ]:
# Train the model for X epochs
fit(train_dl, 50, nn_model, loss_fn, opt)